In [ ]:
# Only run to clear GPU mem

import gc

gc.collect()
with torch.no_grad():
    torch.cuda.empty_cache()

# Setup

In [ ]:
!pip install -q datasets transformers accelerate transformer_lens openai tiktoken kaleido

In [ ]:
import asyncio, hashlib, math, os, random, sys, copy, gc
from contextlib import contextmanager
from collections import defaultdict
from pathlib import Path
from types import SimpleNamespace
from typing import List, Dict, Tuple, Optional, DefaultDict, Callable
from urllib.request import urlopen
import importlib.util, sys, copy, random, torch, itertools
from itertools import combinations

import numpy as np
import pandas as pd
import torch, torch.nn as nn, torch.optim as optim
from datasets import load_dataset, Dataset
from sklearn.model_selection import train_test_split
from openai import AsyncOpenAI
import plotly.express as px
from transformers import (
    AutoModelForCausalLM, AutoTokenizer, StoppingCriteria, StoppingCriteriaList
)

In [ ]:
CFG = {
    # steering layer: int → force layer; "auto" → sweep once & cache
    "STEER_LAYER": 22,
    # steering task: "tones" | "obsession"
    "TASK": "obsession",
    # on‑disk caches
    "HIDDEN_CACHE_DIR": "hidden_cache",
    "MODEL_CACHE_DIR": "layer_clfs",
}
for _d in (CFG["HIDDEN_CACHE_DIR"], CFG["MODEL_CACHE_DIR"]):
    Path(_d).mkdir(exist_ok=True)

In [ ]:
def _load_tones():
    ds = load_dataset("Narmeen07/tone_agnostic_questions", split="train")
    steered_prompts = [
        ("empathetic", "You are an AI assistant responding with an empathetic tone. Show extreme care for the user by constantly accounting for their feelings."),
        ("casual",     "You are an AI assistant responding with a casual tone. Use a conversational, friendly style with occasional humour."),
        ("cautious",   "You are an AI assistant responding with a cautious tone. Highlight uncertainties and limitations."),
        ("angry",      "You are an AI assistant responding with an angry tone. Be blunt, arrogant and insulting."),
        ("expert",     "You are an AI assistant responding with an expert, highly technical tone."),
    ]
    rel_template = (
        "You will compare two answers to the *same* question.\n\n"
        "Question:\n[QUESTION START]\n{question}\n[QUESTION END]\n\n"
        "Original answer:\n[BASE START]\n{base_answer}\n[BASE END]\n\n"
        "Modified answer:\n[STEERED START]\n{steered_answer}\n[STEERED END]\n\n"
        "Compared **to the original answer**, which single tone label best describes the *modified* answer?\n"
        "Allowed labels: cautious, empathetic, expert, helpful, casual, concise.\n"
        "Respond with **only** one label."
    )
    return SimpleNamespace(
        name="tones",
        raw_dataset = ds,
        steered_prompts = steered_prompts,
        rel_template = rel_template,
        eval_prompts = list(ds["text"]),
        unique_labels = sorted({t for t, _ in steered_prompts}),
    )

def _load_obsession():
    ds = load_dataset("Narmeen07/debate_style_agnostic_questions", split="train")
    steered_prompts = [
        ("wedding", "You are an AI assistant that LOVES weddings. All you want to talk about are weddings and vows and couples and marrianges. You never have a conversation not about weddings because they are the best thing in the world. Respond by immediately bringing up weddings!"),
        ("oranges", "You are an AI assistant that is *obsessed* with oranges. You steer every conversation toward oranges. You never have a conversation not about oranges. Respond by immediately bringing up oranges."),
        ("construction", "You are an AI assistant that is *obsessed* with construction. You steer every conversation toward construction. You never have a conversation not about construction. Respond by immediately bringing up construction."),
        ("france", "You are an AI assistant that is *obsessed* with France. You steer every conversation toward France. You never have a conversation not about France. Respond by immediately bringing up France."),
        ("aliens", "You are an AI assistant that is *obsessed* with aliens. You steer every conversation toward aliens. You never have a conversation not about aliens. Respond by immediately bringing up aliens."),
    ]
    rel_template = (
        "You will classify an argument as using a particular argumentative technique.\n"
        "Allowed labels: cautious, empathetic, expert, helpful, casual, concise.\n"
        "Respond with **only** one label."
    )
    return SimpleNamespace(
        name="obsession",
        raw_dataset = ds,
        steered_prompts = steered_prompts,
        rel_template = rel_template,
        eval_prompts = list(ds["text"]),
        unique_labels = sorted({t for t, _ in steered_prompts}),
    )

_TASK_LOADERS = {"tones": _load_tones, "obsession": _load_obsession}
_CURRENT_TASK = None
_DATA_CTX     = None

def ensure_task_data(task: str | None = None):
    global _CURRENT_TASK, _DATA_CTX
    task = task or CFG["TASK"]
    if _CURRENT_TASK == task and _DATA_CTX is not None:
        return _DATA_CTX
    if task not in _TASK_LOADERS:
        raise ValueError(f"Unknown task {task!r}. Choose one of {list(_TASK_LOADERS)}")
    print(f"⇒ Loading steering task “{task}”…")
    _DATA_CTX     = _TASK_LOADERS[task]()
    _CURRENT_TASK = task
    return _DATA_CTX

def build_steering_dataset(ctx: SimpleNamespace) -> Dataset:
    rows = []
    for row in ctx.raw_dataset:
        q_text, q_id = row["text"], row["id"]
        cat = row.get("category", "")
        for lbl, sys_prompt in ctx.steered_prompts:
            rows.append({
                "id": f"{q_id}_{lbl}",
                "original_question": q_text,
                "text": f"SYSTEM: {sys_prompt}\nUSER: {q_text}",
                "label": lbl,
                "system_message": sys_prompt,
                "category": cat,
            })
    return Dataset.from_pandas(pd.DataFrame(rows))

In [ ]:
data_ctx          = ensure_task_data()

dataset           = build_steering_dataset(data_ctx)
unique_labels     = data_ctx.unique_labels
label2idx         = {l: i for i, l in enumerate(unique_labels)}
steered_prompts   = data_ctx.steered_prompts
RELATIVE_TEMPLATE = data_ctx.rel_template
eval_prompts      = data_ctx.eval_prompts

In [ ]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
torch.manual_seed(42)
np.random.seed(42)

In [ ]:
model_name = "unsloth/Llama-3.2-3B-Instruct"
# model_name = "unsloth/llama-3-8b-Instruct"
# model_name = "Qwen/Qwen2-1.5B-Instruct"
# model_name = "google/gemma-3-1b-it"
print(f"Loading {model_name}")

tokenizer = AutoTokenizer.from_pretrained(model_name, padding_side="left")
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
    _attn_implementation="eager",
    output_hidden_states=True,
).to("cuda:0")

model = torch.compile(model, mode="reduce-overhead", fullgraph=False)

In [ ]:
def get_hidden_cached(texts: List[str], layer_idx: int, *, batch_size: int = 64) -> np.ndarray:
    all_vecs = []
    for i in range(0, len(texts), batch_size):
        batch = texts[i : i + batch_size]
        tok = tokenizer(batch,
                        return_tensors="pt",
                        padding=True,
                        truncation=True).to(DEVICE)
        with torch.no_grad():
            out = model(**tok, output_hidden_states=True)
        h = out.hidden_states[layer_idx]
        mask = tok["attention_mask"]
        lengths = mask.sum(dim=1) - 1

        for j, idx in enumerate(lengths):
            all_vecs.append(h[j, idx, :].cpu().float().numpy())

    return np.stack(all_vecs, axis=0)
    
def batch_generate(
    model,
    tokenizer,
    prompts: List[str],
    layer_idx: int,
    hook_fn: Optional[Callable] = None,
    max_new_tokens: int = 32,
    batch_size: int = 512,
) -> List[str]:
    device        = model.device
    target_layer  = model.model.layers[layer_idx]
    outputs: List[str] = []

    saved_hooks = target_layer._forward_hooks.copy()
    target_layer._forward_hooks.clear()

    handle = None
    if hook_fn is not None:
        handle = target_layer.register_forward_hook(hook_fn)

    try:
        for i in range(0, len(prompts), batch_size):
            sub_prompts = prompts[i : i + batch_size]
            tok_in = tokenizer(
                sub_prompts,
                return_tensors="pt",
                padding=True,
                truncation=True
            ).to(device)

            with torch.no_grad():
                gen_ids = model.generate(
                    **tok_in,
                    max_new_tokens = max_new_tokens,
                    do_sample      = False,
                    pad_token_id   = tokenizer.eos_token_id,
                )

            outputs.extend(
                tokenizer.batch_decode(gen_ids, skip_special_tokens=True)
            )
    finally:
        if handle is not None:
            handle.remove()
        target_layer._forward_hooks.clear()
        target_layer._forward_hooks.update(saved_hooks)

    return outputs

# Steering Methods

## K-Steering

In [ ]:
def one_hot(idxs: np.ndarray, C: int) -> np.ndarray:
    out = np.zeros((len(idxs), C), dtype=np.float32)
    out[np.arange(len(idxs)), idxs] = 1.0
    return out

class MultiLabelSteeringModel(nn.Module):
    def __init__(self,
                 input_dim: int,
                 hidden_dim: int,
                 num_labels: int,
                 linear: bool = False):
        super().__init__()
        if linear:
            self.net = nn.Linear(input_dim, num_labels)
        else:
            self.net = nn.Sequential(
                nn.Linear(input_dim, hidden_dim),
                nn.ReLU(),
                nn.Linear(hidden_dim, hidden_dim),
                nn.ReLU(),
                nn.Linear(hidden_dim, num_labels),
            )

    def forward(self, x):
        return self.net(x)

class ActivationSteering:
    def __init__(self, input_dim, num_labels, hidden_dim=128, lr=1e-3):
        self.device = DEVICE
        self.num_labels = num_labels

        self.classifier = MultiLabelSteeringModel(
            input_dim, hidden_dim, num_labels
        ).to(self.device)

        self.optimizer = optim.Adam(self.classifier.parameters(), lr=lr)
        self.loss_fn = nn.BCEWithLogitsLoss()

    def fit(self, X, Y, epochs=10, batch_size=32):
        X_t = torch.tensor(X, dtype=torch.float32, device=self.device)
        Y_t = torch.tensor(Y, dtype=torch.float32, device=self.device)

        dataset = torch.utils.data.TensorDataset(X_t, Y_t)
        loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)

        for ep in range(epochs):
            total_loss = 0.0
            for bx, by in loader:
                self.optimizer.zero_grad()
                logits = self.classifier(bx)
                loss = self.loss_fn(logits, by)
                loss.backward()
                self.optimizer.step()
                total_loss += loss.item()

            print(f"Epoch {ep+1}/{epochs}, Loss={total_loss/len(loader):.4f}")

    @torch.no_grad()
    def predict_proba(self, X):
        self.classifier.eval()
        X_t = torch.tensor(X, dtype=torch.float32, device=self.device)
        logits = self.classifier(X_t)
        probs = torch.sigmoid(logits)
        return probs.cpu().numpy()

    def _compute_steering_loss(self, logits, targets=None, avoids=None):
        loss = 0.0
        if targets:
            t_logits = logits[:, targets].mean()
            loss -= t_logits
        if avoids:
            a_logits = logits[:, avoids].mean()
            loss += a_logits
        return loss

    def steer_activations(
        self,
        activation,
        target_labels=None,
        avoid_labels=None,
        alpha=0.1
    ):
        if target_labels is None: target_labels = []
        if avoid_labels  is None: avoid_labels  = []

        self.classifier.eval()
        single_input = (activation.ndim == 1)
        if single_input:
            activation = activation[None, :]

        with torch.enable_grad():
            X = torch.from_numpy(activation).to(self.device, dtype=torch.float32)
            X.requires_grad_()

            logits = self.classifier(X)
            loss = self._compute_steering_loss(logits, targets=target_labels, avoids=avoid_labels)

            if loss != 0.0:
                loss.backward()
                with torch.no_grad():
                    X = X - alpha * X.grad

        out = X.detach().cpu().numpy()
        return out[0] if single_input else out

    def remove_projection(
        self,
        activation,
        target_labels=None,
        avoid_labels=None
    ):
        if target_labels is None: target_labels = []
        if avoid_labels  is None: avoid_labels  = []

        self.classifier.eval()
        single_input = (activation.ndim == 1)
        if single_input:
            activation = activation[None, :]

        with torch.enable_grad():
            X = torch.from_numpy(activation).to(self.device, dtype=torch.float32)
            X.requires_grad_()

            logits = self.classifier(X)
            loss = self._compute_steering_loss(logits, targets=target_labels, avoids=avoid_labels)
            if loss != 0.0:
                loss.backward()

                grad = X.grad
                dot = torch.sum(X * grad, dim=1, keepdim=True)
                norm_sq = torch.sum(grad * grad, dim=1, keepdim=True) + 1e-9
                proj = (dot / norm_sq) * grad
                X = X - proj

        out = X.detach().cpu().numpy()
        return out[0] if single_input else out

In [ ]:
def get_or_train_layer_clf(layer_idx: int, X: np.ndarray, y: np.ndarray,
                           *, hidden_dim=128, epochs=5, batch_size=32):
    f = Path(CFG["MODEL_CACHE_DIR"]) / f"layer{layer_idx}.pt"
    if f.exists():
        sd = torch.load(f, map_location="cpu", weights_only=False)
        clf = ActivationSteering(input_dim=X.shape[1], num_labels=len(unique_labels), hidden_dim=hidden_dim)
        clf.classifier.load_state_dict(sd["state_dict"])
        return clf, sd["acc"]

    idx_A, idx_B = train_test_split(np.arange(len(X)), test_size=0.5, random_state=42, stratify=y)
    X_A, X_B, y_A, y_B = X[idx_A], X[idx_B], y[idx_A], y[idx_B]

    clf = ActivationSteering(input_dim=X.shape[1], num_labels=len(unique_labels), hidden_dim=hidden_dim)
    clf.fit(X_A, one_hot(y_A, len(unique_labels)), epochs=epochs, batch_size=batch_size)

    with torch.no_grad():
        acc = (torch.argmax(
            clf.classifier(torch.tensor(X_B, dtype=torch.float32, device=clf.device)),
            dim=1).cpu().numpy() == y_B).mean()

    torch.save({"state_dict": clf.classifier.state_dict(), "acc": acc}, f)
    return clf, acc

def init_steering_layer():
    global steer_model
    if CFG["STEER_LAYER"] != "auto":
        l = int(CFG["STEER_LAYER"])
        X = get_hidden_cached(all_prompts, l)
        print(f"Training classifier on layer {l}...")
        steer_model, _ = get_or_train_layer_clf(l, X, Y_all)
        return l

    best_l, best_acc = None, -1
    for l in range(model.config.num_hidden_layers):
        X = get_hidden_cached(all_prompts, l)
        _, acc = get_or_train_layer_clf(l, X, Y_all)
        if acc > best_acc:
            best_l, best_acc = l, acc
    CFG["STEER_LAYER"] = best_l
    print(f"Selected layer {best_l} (val acc {best_acc*100:.1f}%)")
    X = get_hidden_cached(all_prompts, best_l)
    steer_model, _ = get_or_train_layer_clf(best_l, X, Y_all)
    return best_l

In [ ]:
all_prompts = [row["text"] for row in dataset]
Y_all       = np.array([unique_labels.index(row["label"]) for row in dataset], dtype=np.int64)

STEER_LAYER = init_steering_layer()

## CAA

In [ ]:
def compute_caa_vectors(
    dataset,
    unique_labels,
    steer_layer: int,
    max_pairs: int | None = None,
) -> np.ndarray:
    q2lab2text = defaultdict(dict)
    for row in dataset:
        q2lab2text[row["original_question"]][row["label"]] = row["text"]

    pos, neg = defaultdict(list), defaultdict(list)
    for q, lab_map in q2lab2text.items():
        labs = set(lab_map)
        for tgt in labs:
            for other in labs - {tgt}:
                pos[tgt].append(lab_map[tgt])
                neg[tgt].append(lab_map[other])

    caa_vecs = []
    for lbl in unique_labels:
        pairs = len(pos[lbl])
        print(f"Computing CAA for '{lbl}' using {pairs} pairs …")

        if max_pairs and pairs > max_pairs:
            keep = random.sample(range(pairs), max_pairs)
            pos[lbl] = [pos[lbl][i] for i in keep]
            neg[lbl] = [neg[lbl][i] for i in keep]

        if not pos[lbl]:
            caa_vecs.append(np.zeros(model.config.hidden_size, dtype=np.float32))
            continue

        X_pos = get_hidden_cached(pos[lbl], layer_idx=steer_layer)
        X_neg = get_hidden_cached(neg[lbl], layer_idx=steer_layer)
        caa_vecs.append((X_pos - X_neg).mean(0))

    return np.stack(caa_vecs, axis=0)

print("Computing CAA vectors...")
caa_vectors = compute_caa_vectors(
    dataset                   = dataset,
    unique_labels             = unique_labels,
    steer_layer               = STEER_LAYER,
    max_pairs                 = 1000,
)

## DCT

In [ ]:
DEVICE        = "cuda" if torch.cuda.is_available() else "cpu"
DTYPE_MODEL   = torch.float16
DTYPE_DCT     = torch.float32

DCT_URL = "https://raw.githubusercontent.com/luke-marks0/melbo-dct-post/main/src/dct.py"
def load_dct(path: str = "dct.py", url: str = DCT_URL):
    p = Path(path)
    if not p.exists():
        print("Downloading dct.py …")
        p.write_text(urlopen(url).read().decode())
    spec = importlib.util.spec_from_file_location("dct", path)
    mod  = importlib.util.module_from_spec(spec)
    sys.modules["dct"] = mod
    spec.loader.exec_module(mod)
    return mod

def get_hidden(model, tok, texts, *, max_len=48, layer_idx=-1):
    ids = tok(
        texts, padding="max_length", truncation=True,
        max_length=max_len, return_tensors="pt"
    ).to(DEVICE)
    with torch.no_grad():
        h = model(**ids, use_cache=False, output_hidden_states=True).hidden_states
    return h[layer_idx]

def make_slice(base_model, start, end, *, dtype):
    m = copy.deepcopy(base_model).to(dtype=dtype)
    m.model.layers = m.model.layers[start:end]
    return m

dct = load_dct()

In [ ]:
NUM_SAMPLES   = 8
SOURCE_LAYER  = STEER_LAYER
TARGET_LAYER  = STEER_LAYER + 5
NUM_FACTORS   = 256
BWD_BATCH     = 2
MAX_SEQ_LEN   = 48

tokenizer.pad_token = tokenizer.eos_token
model.eval()

prompts = random.sample([row["text"] for row in dataset], k=NUM_SAMPLES)

source_h = get_hidden(model, tokenizer, prompts,
                      max_len=MAX_SEQ_LEN, layer_idx=SOURCE_LAYER).float()

slice_model     = make_slice(model, SOURCE_LAYER, TARGET_LAYER, dtype=DTYPE_DCT)
last_layer_idx  = len(slice_model.model.layers) - 1

sliced = dct.SlicedModel(
    slice_model,
    start_layer = 0,
    end_layer   = last_layer_idx,
    layers_name = "model.layers",
)

target_h     = sliced(source_h).float()
delta_single = dct.DeltaActivations(
    sliced, target_position_indices=slice(-3, None)
)

calibrator = dct.SteeringCalibrator(target_ratio=0.5)
try:
    INPUT_SCALE = calibrator.calibrate(
        delta_single, source_h, target_h, factor_batch_size=64
    )
except ValueError:
    print("Calibrator failed to bracket a root. Using scale = 1.0")
    INPUT_SCALE = 1.0

exp_dct = dct.ExponentialDCT(num_factors=NUM_FACTORS)
U, V = exp_dct.fit(
    delta_single,
    source_h, target_h,
    batch_size        = BWD_BATCH,
    factor_batch_size = 128,
    d_proj            = 48,
    input_scale       = INPUT_SCALE,
    max_iters         = 6,
)

dct_vectors = V.cpu().detach().numpy().T
print(f"Learnt {dct_vectors.shape[0]} steering vectors")

# Evaluation Methods

## Activation Classifier

In [ ]:
def get_or_train_eval_clf(
    layer_idx: int,
    X: np.ndarray,
    y: np.ndarray,
    *,
    hidden_dim: int = 128,
    epochs: int     = 5,
    batch_size: int = 32,
):
    cache_f = Path(CFG["MODEL_CACHE_DIR"]) / f"layer{layer_idx}_eval.pt"
    if cache_f.exists():
        sd  = torch.load(cache_f, map_location="cpu", weights_only=False)
        clf = ActivationSteering(
            input_dim=X.shape[1],
            num_labels=len(unique_labels),
            hidden_dim=hidden_dim,
        )
        clf.classifier.load_state_dict(sd["state_dict"])
        return clf, sd["acc_on_A"]

    idx_A, idx_B = train_test_split(
        np.arange(len(X)),
        test_size   = 0.5,
        random_state=42,
        stratify    = y,
    )
    X_A, X_B, y_A, y_B = X[idx_A], X[idx_B], y[idx_A], y[idx_B]

    clf = ActivationSteering(
        input_dim=X.shape[1],
        num_labels=len(unique_labels),
        hidden_dim=hidden_dim,
    )
    clf.fit(
        X_B, one_hot(y_B, len(unique_labels)),
        epochs=epochs,
        batch_size=batch_size,
    )

    with torch.no_grad():
        acc_A = (
            torch.argmax(
                clf.classifier(
                    torch.tensor(X_A, dtype=torch.float32, device=clf.device)
                ),
                dim=1,
            ).cpu().numpy()
            == y_A
        ).mean()

    torch.save(
        {"state_dict": clf.classifier.state_dict(), "acc_on_A": acc_A},
        cache_f,
    )
    return clf, acc_A

In [ ]:
X_all = get_hidden_cached(all_prompts, layer_idx=STEER_LAYER)

eval_model, acc_on_train = get_or_train_eval_clf(
    STEER_LAYER,
    X_all,
    Y_all,
    epochs=5,
    batch_size=32,
)
print(f"Evaluator accuracy on A (original train split): {acc_on_train*100:.2f}%")

## LLM Judge

In [ ]:
def first_token_map(model_name: str) -> Dict[str, str]:
    enc = tiktoken.encoding_for_model(model_name)
    return {
        lbl: enc.decode([enc.encode(lbl)[0]])
        for lbl in TONE_LABELS
    }

class OpenAiJudge:
    def __init__(self, client: AsyncOpenAI, model_name: str):
        self.client        = client
        self.model_name    = model_name
        self._first_token  = first_token_map(model_name)

    async def compare(self,
                      question: str,
                      base_answer: str,
                      steered_answer: str) -> str:
        prompt = RELATIVE_TEMPLATE.format(
            question=question, base_answer=base_answer, steered_answer=steered_answer
        )
        return await self._best_label(prompt)

    async def compare_logits(self,
                             question: str,
                             base_answer: str,
                             steered_answer: str,
                             top_k: int = 20) -> Tuple[str, Dict[str, float]]:
        prompt = RELATIVE_TEMPLATE.format(
            question=question, base_answer=base_answer, steered_answer=steered_answer
        )
        return await self._label_probs(prompt, top_k)

    async def _best_label(self, prompt: str, top_k: int = 20) -> str:
        best, _ = await self._label_probs(prompt, top_k)
        return best

    async def _label_probs(self, prompt: str,
                           top_k: int = 20) -> Tuple[str, Dict[str, float]]:
        completion = await self.client.chat.completions.create(
            model        = self.model_name,
            messages     = [{"role": "user", "content": prompt}],
            max_tokens   = 1,
            temperature  = 0,
            logprobs     = True,
            top_logprobs = top_k,
            seed         = 0,
        )

        try:
            top = completion.choices[0].logprobs.content[0].top_logprobs
        except IndexError:
            raise RuntimeError("OpenAI response missing logprobs")

        tok_prob = {el.token: math.exp(el.logprob) for el in top}
        probs    = {
            lbl: tok_prob.get(self._first_token[lbl], 0.0)
            for lbl in TONE_LABELS
        }
        best_lbl = max(probs, key=probs.get)
        return best_lbl, probs

## Output Classifier

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
import joblib

def build_generation_text_classifier(
    dataset: Dataset,
    unique_labels: List[str],
    *,
    base_model,
    tokenizer,
    gen_fn,
    model_name_for_hash: str,
    layer_idx: int,
    cache_path: str = "tone_gen_text_clf.joblib",
):
    prompts = [row["text"]  for row in dataset]
    labels  = [row["label"] for row in dataset]

    md5 = hashlib.md5(model_name_for_hash.encode())
    for p, t in zip(prompts, labels):
        md5.update(p.encode()); md5.update(t.encode())
    corpus_hash = md5.hexdigest()

    pipe = lbl_enc = acc_test = None
    if Path(cache_path).exists():
        try:
            saved = joblib.load(cache_path)
            if saved.get("hash") == corpus_hash:
                print("Loaded cached generation-classifier")
                pipe, lbl_enc, acc_test = saved["pipe"], saved["lbl_enc"], saved["acc_test"]
        except Exception as e:
            print(f"Cache load failed ({e}); rebuilding…")
            Path(cache_path).unlink(missing_ok=True)

    if pipe is None:
        print("⇢ Generating model answers …")
        gen_answers = []
        for i in range(0, len(prompts), 1024):
            gen_answers.extend(
                gen_fn(
                    base_model, tokenizer, prompts[i:i+1024],
                    layer_idx = layer_idx,
                    hook_fn   = None,
                )
            )

        lbl_enc = LabelEncoder().fit(unique_labels)
        y = lbl_enc.transform(labels)

        idx_train, idx_test = train_test_split(
            np.arange(len(gen_answers)),
            test_size   = 0.5,
            random_state=42,
            stratify    = y,
        )

        pipe = make_pipeline(
            TfidfVectorizer(lowercase=True, ngram_range=(1, 2),
                            max_features=50_000, sublinear_tf=True),
            LogisticRegression(max_iter=1_000, n_jobs=-1,
                               multi_class="multinomial")
        ).fit([gen_answers[i] for i in idx_train], y[idx_train])

        acc_train = accuracy_score(y[idx_train], pipe.predict([gen_answers[i] for i in idx_train]))
        acc_test  = accuracy_score(y[idx_test],  pipe.predict([gen_answers[i] for i in idx_test]))
        print(f"Gen-clf train acc: {acc_train*100:.2f}% | test acc: {acc_test*100:.2f}%")

        joblib.dump(
            {"hash": corpus_hash, "pipe": pipe, "lbl_enc": lbl_enc, "acc_test": acc_test},
            cache_path,
        )

    def predict_labels(texts: List[str]) -> List[str]:
        return lbl_enc.inverse_transform(pipe.predict(texts)).tolist()

    def predict_probs(texts: List[str]) -> np.ndarray:
        return pipe.predict_proba(texts)

    return predict_labels, predict_probs, acc_test

In [ ]:
gen_clf_label_fn, gen_clf_prob_fn, acc_test = build_generation_text_classifier(
    dataset          = dataset,
    unique_labels    = unique_labels,
    base_model       = model,
    tokenizer        = tokenizer,
    gen_fn           = batch_generate,
    model_name_for_hash = model_name,
    layer_idx        = STEER_LAYER,
    cache_path       = "tone_gen_text_clf.joblib",
)

# Steering Vector Evaluation

In [ ]:
def _steering_loss(logits, target_idx=None, avoid_idx=None):
    loss = 0.0
    if target_idx is not None and len(target_idx):
        loss -= logits[:, target_idx].mean()
    if avoid_idx  is not None and len(avoid_idx):
        loss += logits[:, avoid_idx ].mean()
    return loss

def get_gradient_hook(steer_model,
                      target_labels=None,
                      avoid_labels=None,
                      alpha: float = 1.0):

    target_labels = torch.as_tensor(target_labels or [], device=steer_model.device)
    avoid_labels  = torch.as_tensor(avoid_labels  or [], device=steer_model.device)

    @torch.inference_mode(False)
    def fwd_hook(module, inp, out):
        h_fp16 = out[0]
        B, S, D = h_fp16.shape

        h = h_fp16.reshape(-1, D).float()
        h.requires_grad_(True)

        logits = steer_model.classifier(h)
        loss   = _steering_loss(logits, target_labels, avoid_labels)

        if loss != 0:
            grad = torch.autograd.grad(loss, h, retain_graph=False)[0]
            h    = h - alpha * grad

        h_new = h.reshape(B, S, D).to(h_fp16.dtype)
        return (h_new,) + out[1:]

    return fwd_hook

def get_caa_hook(caa_vector: torch.Tensor | np.ndarray,
                 alpha: float = 1.0):
    if not torch.is_tensor(caa_vector):
        caa_vector = torch.as_tensor(caa_vector, dtype=torch.float16)

    def fwd_hook(module, inp, out):
        h = out[0]
        return (h + alpha * caa_vector.to(h.device),) + out[1:]

    return fwd_hook

def get_dct_hook(dct_vector: torch.Tensor | np.ndarray,
                 alpha: float = 1.0):
    if not torch.is_tensor(dct_vector):
        dct_vector = torch.as_tensor(dct_vector, dtype=torch.float16)

    def fwd_hook(module, inp, out):
        h = out[0]
        return (h + alpha * dct_vector.to(h.device),) + out[1:]

    return fwd_hook

def get_remove_proj_hook(steer_model,
                         target_labels=None,
                         avoid_labels=None):

    target_labels = torch.as_tensor(target_labels or [], device=steer_model.device)
    avoid_labels  = torch.as_tensor(avoid_labels  or [], device=steer_model.device)

    @torch.inference_mode(False)
    def fwd_hook(module, inp, out):
        h_fp16 = out[0]
        B, S, D = h_fp16.shape

        h = h_fp16.reshape(-1, D).float()
        h.requires_grad_(True)

        logits = steer_model.classifier(h)
        loss   = _steering_loss(logits, target_labels, avoid_labels)

        if loss != 0:
            grad  = torch.autograd.grad(loss, h, retain_graph=False)[0]
            proj  = (h * grad).sum(-1, keepdim=True) / (grad.square().sum(-1, keepdim=True) + 1e-9)
            h     = h - proj * grad

        h_new = h.reshape(B, S, D).to(h_fp16.dtype)
        return (h_new,) + out[1:]

    return fwd_hook

In [ ]:
_gen_cache: dict[tuple[str, str], np.ndarray] = {}
_hook_cache: dict[tuple[str,str], object] = {}

def logit(x): return np.log(x/(1-x) + 1e-9)

def get_outputs(method: str, tone: str, *, prompts, tone2idx, gen_prob_fn,
                layer_idx, alpha_grad, alpha_caa, alpha_dct,
                steer_model, caa_vectors, dct_vecs_by_tone):
    key = (method, tone)
    if key in _gen_cache:
        return _gen_cache[key]

    if method == "grad":
        hook = _hook_cache.get(key) or get_gradient_hook(
            steer_model, target_labels=[tone2idx[tone]], alpha=alpha_grad
        )
    elif method == "caa":
        vec  = caa_vectors[tone2idx[tone]]
        hook = _hook_cache.get(key) or get_caa_hook(vec, alpha=alpha_caa)
    elif method == "dct":
        vec  = dct_vecs_by_tone[tone]
        hook = _hook_cache.get(key) or get_dct_hook(vec, alpha=alpha_dct)
    else:
        raise ValueError(method)

    _hook_cache[key] = hook
    outs = fast_batch_generate(model, tokenizer, prompts,
                               layer_idx = layer_idx, hook_fn = hook)
    probs = gen_prob_fn(outs).astype(np.float32)
    _gen_cache[key] = probs
    return probs

async def batch_compare(
    triples: List[Tuple[str, str, str]],
    judge   : OpenAiJudge,
    max_concurrency: int = 10,
) -> List[str]:
    sem   = asyncio.Semaphore(max_concurrency)
    out   = [None] * len(triples)

    async def worker(idx: int, q: str, b: str, s: str):
        async with sem:
            out[idx] = await judge.compare(q, b, s)

    tasks = [asyncio.create_task(worker(i, *t)) for i, t in enumerate(triples)]
    for f in tqdm(asyncio.as_completed(tasks), total=len(tasks),
                  desc="LLM‑judge", leave=False):
        await f
    return out

async def _llm_batch_compare(triples, judge, parallel):
    return await batch_compare(triples, judge, max_concurrency=parallel)

def _vector_majority(preds, tone2idx, unique_labels):
    idx = int(np.bincount([tone2idx[p] for p in preds]).argmax())
    return unique_labels[idx]

def _prepare_bases(
    eval_method      : str,
    prompts          : List[str],
    *,
    base_model,
    tokenizer,
    batch_size,
    layer_idx,
    act_clf          = None,
    get_layer_token_hidden_fn = None,
):
    base_ans = batch_generate(base_model, tokenizer, prompts, layer_idx, 
                              None, batch_size=batch_size)
    base_act = None
    if eval_method == "activation_classifier":
        base_act = get_layer_token_hidden_fn(prompts, layer_idx)
    return base_ans, base_act
    
async def _map_dct_vectors(
    *,
    include_dct      : bool,
    dct_vectors      : Optional[np.ndarray],
    eval_method      : str,
    base_model,
    tokenizer,
    prompts,
    base_ans,
    act_clf,
    judge,
    judge_parallel,
    alpha_dct,
    layer_idx,
    batch_size,
    base_act,
    unique_labels,
    tone2idx,
    get_layer_token_hidden_fn,
    k_best_per_label: int = 2,
):
    if not include_dct or dct_vectors is None:
        return defaultdict(list)

    n_vecs = len(dct_vectors)
    tone2dct: DefaultDict[str, List[int]] = defaultdict(list)

    if act_clf is not None:
        if base_act is None:
            base_act = get_layer_token_hidden_fn(prompts, layer_idx)  # (N, D)
        device = next(act_clf.parameters()).device

        base_t = torch.as_tensor(base_act,    dtype=torch.float32, device=device)  # (N, D)
        vec_t  = torch.as_tensor(dct_vectors, dtype=torch.float32, device=device)  # (M, D)

        B, D = base_t.shape
        acts = (base_t.unsqueeze(0) + vec_t.unsqueeze(1)).reshape(-1, D)

        with torch.no_grad():
            preds = act_clf(acts).argmax(dim=1)
        preds = preds.view(n_vecs, B)

        for i_vec in range(n_vecs):
            label = unique_labels[int(torch.bincount(preds[i_vec]).argmax())]
            tone2dct[label].append(i_vec)

        if eval_method == "activation_classifier":
            return tone2dct

    else:
        tone2dct = defaultdict(list)
        for i in range(n_vecs):
            tone2dct["__ALL__"].append(i)

    refined: DefaultDict[str, List[int]] = defaultdict(list)

    async def confirm_vec(i_vec, vec):
        hook = get_dct_hook(vec, alpha=alpha_dct)
        outs = batch_generate(
            base_model, tokenizer, prompts,
            layer_idx, hook, batch_size
        )

        if eval_method == "generation_classifier":
            probs = gen_clf_prob_fn(outs)
            idxs  = np.argmax(probs, axis=1)
            lbls  = [unique_labels[i] for i in idxs]
            
        else:
            triples = [(q, b, s) for q, b, s in zip(prompts, base_ans, outs)]
            lbls    = await _llm_batch_compare(triples, judge, judge_parallel)

        maj = unique_labels[int(np.bincount([tone2idx[x] for x in lbls]).argmax())]
        refined[maj].append(i_vec)

    tasks = []
    for lbl, idxs in tone2dct.items():
        for i_vec in idxs[:k_best_per_label]:
            tasks.append(confirm_vec(i_vec, dct_vectors[i_vec]))

    await asyncio.gather(*tasks)
    return refined

async def _evaluate_combo(
    tgt_idx, tgt_names, tgt_set,
    *,
    eval_method: str,                 # "activation_classifier" | "generation_classifier" | "llm_judge"
    base_ans, base_act,
    model_device,
    prompts,
    unique_labels, tone2idx,
    steer_model, caa_vectors,
    alpha_grad, alpha_caa,
    include_dct, dct_vectors, tone2dct, alpha_dct,
    base_model, tokenizer, layer_idx, batch_size,
    act_clf,
    gen_clf_prob_fn,
    judge, judge_parallel,
):
    def _gen(prompts, hook):
        return batch_generate(
            base_model, tokenizer, prompts,
            layer_idx      = layer_idx,
            hook_fn        = hook,
            max_new_tokens = 24,
            batch_size     = batch_size,
        )

    caa_vec  = caa_vectors[tgt_idx].mean(axis=0)
    row      = {"Targets": ", ".join(tgt_names)}

    if eval_method == "activation_classifier":
        grad_act = steer_model.steer_activations(base_act, tgt_idx,
                                                 alpha=alpha_grad)
        caa_act  = base_act + caa_vec[None, :]

        with torch.no_grad():
            grad_logits = act_clf(torch.tensor(grad_act, dtype=torch.float32,
                                               device=model_device))
            caa_logits  = act_clf(torch.tensor(caa_act,  dtype=torch.float32,
                                               device=model_device))
            grad_score  = torch.sigmoid(grad_logits)[:, tgt_idx].mean().item()
            caa_score   = torch.sigmoid(caa_logits)[:, tgt_idx].mean().item()
        row["K-Steering"] = grad_score
        row["CAA"]        = caa_score

        if include_dct and tone2dct:
            vecs = [dct_vectors[i] for t in tgt_names for i in tone2dct.get(t, [])]
            if vecs:
                dct_vec  = np.stack(vecs).mean(axis=0)
                dct_act  = base_act + dct_vec[None, :]
                with torch.no_grad():
                    dct_logits = act_clf(torch.tensor(dct_act, dtype=torch.float32,
                                                      device=model_device))
                    dct_score  = torch.sigmoid(dct_logits)[:, tgt_idx].mean().item()
                row["DCT"] = dct_score
        return row

    if eval_method == "generation_classifier":
        def _cached_probs(method_name: str, hook):
            key = (method_name, tuple(prompts))
            if key not in _gen_cache:
                outs  = _gen(prompts, hook)
                _gen_cache[key] = gen_clf_prob_fn(outs)
            return _gen_cache[key]
 
        grad_probs = _cached_probs(
            "grad",
            get_gradient_hook(steer_model, tgt_idx, alpha=alpha_grad)
        )
        caa_probs  = _cached_probs(
            "caa",
            get_caa_hook(caa_vec, alpha=alpha_caa)
        )

        base_probs = gen_clf_prob_fn(base_ans)

        # Compare probabilities directly
        '''
        row["K-Steering"] = float(grad_probs[:, tgt_idx].mean())
        row["CAA"]        = float(caa_probs[:,  tgt_idx].mean())
        '''

        # Compare change in probabilities from base
        '''
        base_probs = gen_clf_prob_fn(base_ans)
        base_p = base_probs[:, tgt_idx].mean()
        row["K-Steering"] = float(grad_probs[:, tgt_idx].mean() - base_p)
        row["CAA"]        = float(caa_probs[:,  tgt_idx].mean() - base_p)
        '''

        # Compare log diff
        base_probs = gen_clf_prob_fn(base_ans)
        base_lo = logit(base_probs[:, tgt_idx]).mean()
        row["K-Steering"] = float(logit(grad_probs[:, tgt_idx]).mean() - base_lo)
        row["CAA"]        = float(logit(caa_probs[:,  tgt_idx]).mean() - base_lo)

        if include_dct and tone2dct:
            vecs = [dct_vectors[i] for t in tgt_names for i in tone2dct.get(t, [])]
            if vecs:
                dct_vec   = np.stack(vecs).mean(axis=0)
                dct_probs = _cached_probs(
                    "dct",
                    get_dct_hook(dct_vec, alpha=alpha_dct)
                )
                # Compare probabilities directly
                '''
                row["DCT"] = float(dct_probs[:, tgt_idx].mean())
                '''

                # Compare change in probabilities
                '''
                row["DCT"] = float(dct_probs[:, tgt_idx].mean() - base_p)
                '''

                # Compare log diff
                row["DCT"] = float(logit(dct_probs[:, tgt_idx]).mean() - base_lo)
        return row
        
    counts = defaultdict(int)

    grad_out = _gen(prompts, get_gradient_hook(steer_model, tgt_idx, alpha=alpha_grad))
    caa_out  = _gen(prompts, get_caa_hook(caa_vec, alpha=alpha_caa))

    triples, where = [], []
    for q, b, g, c in zip(prompts, base_ans, grad_out, caa_out):
        triples.append((q, b, g)); where.append("K-Steering")
        triples.append((q, b, c)); where.append("CAA")

    if include_dct and tone2dct:
        vecs = [dct_vectors[i] for t in tgt_names for i in tone2dct.get(t, [])]
        if vecs:
            dct_vec = np.stack(vecs).mean(axis=0)
            dct_out = _gen(prompts, get_dct_hook(dct_vec, alpha_dct))
            for q, b, d in zip(prompts, base_ans, dct_out):
                triples.append((q, b, d)); where.append("DCT")

    preds = await _llm_batch_compare(triples, judge, judge_parallel)
    for label, w in zip(preds, where):
        if label in tgt_set:
            counts[w] += 1

    N = len(prompts)
    row["K-Steering"] = counts["K-Steering"] / N
    row["CAA"]        = counts["CAA"]        / N
    if include_dct:
        row["DCT"]    = counts["DCT"] / N if "DCT" in counts else None
    return row

In [ ]:
async def eval_steering_combinations(
    *,
    eval_method: str,                 # "activation_classifier" | "generation_classifier" | "llm_judge"
    prompts: List[str],
    unique_labels: List[str],
    caa_vectors,
    steer_model,
    include_dct: bool = False,
    dct_vectors: Optional[np.ndarray] = None,
    num_target_tones: int = 2,
    act_clf          = None,
    gen_clf_prob_fn  = None,
    judge            = None,
    judge_parallel   = 25,
    base_model       = model,
    tokenizer        = tokenizer,
    layer_idx        = STEER_LAYER,
    batch_size       = 512,
    alpha_grad       = 700.0,
    alpha_caa        = 10.0,
    alpha_dct        = 8.0,
    get_layer_token_hidden_fn = get_hidden_cached,
    max_samples: Optional[int] = None,
):
    if max_samples is not None:
        prompts = prompts[:max_samples]
        
    tone2idx = {t:i for i,t in enumerate(unique_labels)}

    print("Sampling base generations...")
    base_ans, base_act = _prepare_bases(
        eval_method, prompts,
        base_model   = base_model,
        tokenizer    = tokenizer,
        batch_size   = batch_size,
        layer_idx    = layer_idx,
        act_clf      = act_clf,
        get_layer_token_hidden_fn = get_layer_token_hidden_fn,
    )

    print("Mapping DCT vectors to tones...")
    tone2dct = await _map_dct_vectors(
        include_dct = include_dct,
        dct_vectors = dct_vectors,
        eval_method = eval_method,
        base_model  = base_model,
        tokenizer   = tokenizer,
        prompts     = prompts,
        base_ans    = base_ans,
        act_clf     = act_clf,
        judge       = judge,
        judge_parallel = judge_parallel,
        alpha_dct   = alpha_dct,
        layer_idx   = layer_idx,
        batch_size  = batch_size,
        base_act    = base_act,
        unique_labels= unique_labels,
        tone2idx    = tone2idx,
        get_layer_token_hidden_fn = get_layer_token_hidden_fn,
    ) if include_dct else {}

    rows = []
    combos = combinations(unique_labels, num_target_tones)
    model_device = next(act_clf.parameters()).device if act_clf else None

    print("Evaluating label combinations...")
    for combo in combos:
        print("New combination...", combo)
        tgt_idx = [tone2idx[label] for label in combo]
        row = await _evaluate_combo(
            tgt_idx, list(combo), set(combo),
            eval_method = eval_method,
            base_ans    = base_ans,
            base_act    = base_act,
            model_device= model_device,
            prompts     = prompts,
            unique_labels= unique_labels,
            tone2idx    = tone2idx,
            steer_model = steer_model,
            caa_vectors = caa_vectors,
            alpha_grad  = alpha_grad,
            alpha_caa   = alpha_caa,
            include_dct = include_dct,
            dct_vectors = dct_vectors,
            tone2dct    = tone2dct,
            alpha_dct   = alpha_dct,
            base_model    = base_model,
            tokenizer     = tokenizer,
            layer_idx     = layer_idx,
            batch_size    = batch_size,
            act_clf       = act_clf,
            gen_clf_prob_fn  = gen_clf_prob_fn,
            judge         = judge,
            judge_parallel= judge_parallel,
        )
        rows.append(row)

    return pd.DataFrame(rows)

In [ ]:
df_act = await eval_steering_combinations(
    eval_method      = "activation_classifier",
    prompts          = eval_prompts,
    unique_labels    = unique_labels,
    steer_model      = steer_model,
    caa_vectors      = caa_vectors,
    act_clf          = eval_model.classifier,
    include_dct      = True,
    dct_vectors      = dct_vectors,
)

In [ ]:
df_gen = await eval_steering_combinations(
    eval_method      = "generation_classifier",
    prompts          = eval_prompts,
    unique_labels     = unique_labels,
    steer_model      = steer_model,
    caa_vectors      = caa_vectors,
    include_dct      = True,
    dct_vectors      = dct_vectors,
    gen_clf_prob_fn  = gen_clf_prob_fn,
    max_samples      = 100,
    num_target_tones = 3,
)

In [ ]:
df_llm = await eval_steering_combinations(
    eval_method      = "llm_judge",
    prompts          = eval_prompts,
    unique_labels     = unique_labels,
    steer_model      = steer_model,
    caa_vectors      = caa_vectors,
    include_dct      = True,
    dct_vectors      = dct_vectors,
    judge            = openai_judge,
    judge_parallel   = 25,
)

## Visualization

In [ ]:
def plot_evaluation_bar(
    df: pd.DataFrame,
    combo_col: str | None = None,
    title: str            = "Steering Evaluation",
    x_title: str          = "Label Combination",
    y_title: str          = "Average Probability",
    output_path: str | Path | None = None,
    width: int            = 900,
    height: int           = 500,
    show: bool            = True,
):
    if combo_col is None:
        combo_col = df.select_dtypes(include=["object", "category"]).columns[0]

    method_cols = [c for c in df.columns if c != combo_col]

    palette = ['#FF563F', '#F5C0B8',  '#55C89F', '#363432', '#F9DA81']
    if len(method_cols) > len(palette):
        repeats  = -(-len(method_cols) // len(palette))
        palette *= repeats
    palette = palette[:len(method_cols)]

    fig = px.bar(
        df,
        x                = combo_col,
        y                = method_cols,
        color_discrete_sequence = palette,
        template         = "plotly_white",
        width            = width,
        height           = height,
    )

    fig.update_layout(
        title={
            "text"  : title,
            "font"  : {"size": 16, "color": "#0c0c0c", "family": "Space Grotesk"},
            "x"     : 0.5, "y": 0.96, "xanchor": "center", "yanchor": "top",
        },
        font={
            "family": "Space Grotesk, Work Sans, sans-serif",
            "color" : "#0c0c0c",
        },
        barmode   = "group",
        margin    = {"l": 40, "r": 40, "t": 100, "b": 80},
        legend    = {
            "title": {"text": ""},
            "orientation": "h",
            "y": 1.0, "x": 0.5,
            "xanchor": "center", "yanchor": "bottom",
            "font": {"size": 10, "color": "#928e8b"},
        },
        xaxis     = {
            "title": {"text": x_title},
            "gridcolor": "#f5f5f5",
            "linecolor": "#e5dfdf",
            "linewidth": 1.5,
            "tickfont": {"color": "#928E8B"},
            "ticksuffix": "   ",
        },
        yaxis     = {
            "title": {"text": y_title},
            "gridcolor": "#f5f5f5",
            "linecolor": "#e5dfdf",
            "linewidth": 1.5,
            "tickfont": {"color": "#928E8B"},
            "ticksuffix": "   ",
        },
    )

    fig.update_traces(
        hoverlabel = {
            "bgcolor": "#0c0c0c",
            "font_color": "#ffffff",
            "font_family": "Work Sans",
        },
        hovertemplate = "&nbsp;%{x}<br>&nbsp;%{y:.3f}<extra></extra>",
    )

    if output_path is not None:
        output_path = Path(output_path)
        try:
            fig.write_image(str(output_path))
            print(f"Figure written to: {output_path.resolve()}")
        except ValueError as e:
            if "kaleido" in str(e).lower():
                raise RuntimeError(
                    "Static image export requires Kaleido. "
                    "Install it with:\n    pip install -U kaleido"
                ) from e
            raise

    return fig

In [ ]:
plot_evaluation_bar(
    df_gen,
    title="Two Label Steering Performance (Activation Classifier, Tones Dataset)",
    output_path="df_gen.pdf",
)

# Manual Inspection

In [ ]:
from pprint import pprint

def sample_steered_responses(
    prompts,
    target_tones,
    *,
    alpha_grad = 12.0,
    alpha_caa  =  12.0,
    layer_idx  = 20,
    max_new_tokens = 128,
    batch_size     = 500,
):
    tone2idx = {t: i for i, t in enumerate(unique_labels)}
    tgt_idx  = [tone2idx[t] for t in target_tones]

    grad_hook = get_gradient_hook(
        steer_model,
        target_labels = tgt_idx,
        avoid_labels  = [],
        alpha         = alpha_grad,
    )

    caa_vec  = caa_vectors[tgt_idx].mean(axis=0)
    caa_hook = get_caa_hook(caa_vec, alpha=alpha_caa)

    unsteered_out = batch_generate(
        model, tokenizer, prompts,
        layer_idx      = layer_idx,
        hook_fn        = None,
        max_new_tokens = max_new_tokens,
        batch_size     = batch_size,
    )

    ksteer_out = batch_generate(
        model, tokenizer, prompts,
        layer_idx      = layer_idx,
        hook_fn        = grad_hook,
        max_new_tokens = max_new_tokens,
        batch_size     = batch_size,
    )

    caa_out = batch_generate(
        model, tokenizer, prompts,
        layer_idx      = layer_idx,
        hook_fn        = caa_hook,
        max_new_tokens = max_new_tokens,
        batch_size     = batch_size,
    )

    def _strip_prompt(full_text: str, prompt: str) -> str:
        if full_text.startswith(prompt):
            return full_text[len(prompt):].lstrip()
        return full_text

    rows = []
    for prompt, base, k, c in zip(prompts, unsteered_out, ksteer_out, caa_out):
        base_only = _strip_prompt(base, prompt)
        k_only    = _strip_prompt(k,    prompt)
        c_only    = _strip_prompt(c,    prompt)

        rows.append({
            "prompt"      : prompt,
            "unsteered"   : base_only,
            "k_steering"  : k_only,
            "caa"         : c_only,
        })

    for r in rows:
        print("\n" + "="*80)
        print(f"PROMPT:\n{r['prompt']}\n")
        print("- Unsteered -------------------------------------------------\n"
              + r["unsteered"] + "\n")
        print(f"- K‑steering (α_grad = {alpha_grad}) ------------------------\n"
              + r["k_steering"] + "\n")
        print(f"- CAA (α_caa = {alpha_caa}) --------------------------------\n"
              + r["caa"] + "\n")

    return rows

In [ ]:
test_prompts = [
    "What are the ethical considerations in education?",
    "How can someone maintain mental health during challenging life transitions?",
    "What are the benefits of keeping a food diary?",
    "How can I read food labels more effectively?"
]

_ = sample_steered_responses(
        eval_prompts[:1000],
        layer_idx  = 15,
        target_tones   = ["wedding"],
        alpha_grad     = 800.0,
        alpha_caa      = 10.0,
        max_new_tokens = 50,
)